In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
from pathlib import Path
from typing import Tuple, List, Callable

import numpy as np
import pandas as pd
import torch
import glob
from datetime import datetime, timedelta
from pyproj import Geod
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans

import folium
import folium.plugins

In [ ]:
wgs84_geod = Geod(ellps='WGS84') # Distance will be measured in meters on this ellipsoid - more accurate than a spherical method

def coords_to_areas(target): # Calculate to which area an opening's coordinates (target) "belong to"
    _,_,dist = wgs84_geod.inv(virtual_area_centers['GPS_Latitude'], virtual_area_centers['GPS_Longitude'], np.full(len(dist),target['GPS_Latitude']), np.full(len(dist), target['GPS_Longitude']))
    return pd.Series((1 - dist / sum(dist)) / (len(dist) - 1)) # Percentage of how much an opening belongs to each area

In [ ]:
swlat=55.4355410101663
swlon=12.140848911388979
nelat=56.06417055142977
nelon=12.688363746232875
rent_files = glob.glob(str((Path.cwd().parent / 'data' / 'raw' / 'SN rentals' / '*.xlsx')))
rent_dfs = [pd.read_excel(f, skiprows=[0,1]) for f in rent_files]
rental = pd.concat(rent_dfs,ignore_index=True)
rental = rental[rental['[Partner_Rental_ID]']!='[Partner_Rental_ID]']
rental.columns = rental.columns.str.replace("[","", regex=False)
rental.columns = rental.columns.str.replace("]","", regex=False)
rental = rental.loc[:, ['Vehicle_Number_Plate', 'Vehicle_Engine_Type',
'Vehicle_Model', 'Revenue_Net',
'Start_Datetime_Local', 'End_Datetime_Local',
'Start_GPS_Latitude', 'Start_GPS_Longitude',
'End_GPS_Latitude', 'End_GPS_Longitude', 'Package_Description',
'Operation_State_Name_Before', 'Operation_State_Name_After', 'Reservation_YN',
'Prebooking_YN', 'Servicedrive_YN', 'Start_Zone_Name', 'End_Zone_Name']]
# Filter rentals outside of analysis zone
rental = rental[
    (rental['Start_GPS_Latitude'] > swlat) & (rental['Start_GPS_Latitude'] < nelat) & 
    (rental['Start_GPS_Longitude'] > swlon) & (rental['Start_GPS_Longitude'] < nelon) &
    (rental['End_GPS_Latitude'] > swlat) & (rental['End_GPS_Latitude'] < nelat) & 
    (rental['End_GPS_Longitude'] > swlon) & (rental['End_GPS_Longitude'] < nelon)]

In [ ]:
km = KMeans(n_clusters=20, verbose=1).fit(rental.loc[:,['Start_GPS_Latitude','Start_GPS_Longitude']])
areas = pd.DataFrame(km.cluster_centers_, columns=['GPS_Latitude','GPS_Longitude'])
rental.loc[:,'Virtual_Start_Zone_Name'] = km.labels_
rental.loc[:,'Virtual_End_Zone_Name'] = [label for label in km.predict(rental.loc[:,['End_GPS_Latitude','End_GPS_Longitude']])]

In [ ]:
virtual_area_centers=areas

In [ ]:
rental = pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'rental.csv', low_memory=False)
openings = pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'openings.csv')
virtual_area_centers = pd.read_csv(Path.cwd().parent / 'data' / 'processed' / 'areas.csv', index_col=0)

In [ ]:
# Area centers based on current areas
area_centers = rental.groupby('Start_Zone_Name').mean()[['Start_GPS_Latitude','Start_GPS_Longitude']]
area_centers.rename(columns={
    'Start_GPS_Latitude': 'GPS_Latitude', 
    'Start_GPS_Longitude': 'GPS_Longitude'}, inplace=True)
area_centers.index.names = ['Area']

In [ ]:
openings['Created_Datetime_Local'] = pd.to_datetime(openings['Created_Datetime_Local'], format='%Y-%m-%d %H:%M')
openings = pd.get_dummies(openings, columns=['Platform'], drop_first=True)

In [ ]:
rental['Start_Datetime_Local'] = pd.to_datetime(rental['Start_Datetime_Local'], format='%Y-%m-%d %H:%M')
rental['End_Datetime_Local'] = pd.to_datetime(rental['End_Datetime_Local'], format='%Y-%m-%d %H:%M')
rental = pd.get_dummies(rental, columns=['Vehicle_Engine_Type'], drop_first=True)
rental = pd.get_dummies(rental, columns=['Vehicle_Model'])
one_hot_zones = pd.get_dummies(rental.loc[:,['Virtual_Start_Zone_Name', 'Virtual_End_Zone_Name']], columns=['Virtual_Start_Zone_Name', 'Virtual_End_Zone_Name'])
rental = pd.concat([rental, one_hot_zones], axis=1)

In [ ]:
time_start = max(rental['Start_Datetime_Local'].min(), openings['Created_Datetime_Local'].min())
time_end = min(rental['End_Datetime_Local'].max(), openings['Created_Datetime_Local'].max())
print('Time limits:', time_start, 'to', time_end)
total_time = time_end-time_start

In [ ]:
ratio = []
for _ in tqdm(range(100)):
    timepoint = time_start + timedelta(seconds=total_time.total_seconds()*random.uniform(0,1))
    #filtered_rentals = rental[rental['End_Datetime_Local'] <= timepoint].drop('Revenue_Net', axis=1)
    #filtered_rentals = filtered_rentals.sort_values(by='End_Datetime_Local').drop_duplicates(subset='Vehicle_Number_Plate', keep='last') # Keep the last location
    current_trips = rental[(rental['Start_Datetime_Local'] <= timepoint) & (rental['End_Datetime_Local'] > timepoint) & (rental['Servicedrive_YN']==1)] # Cars in use
    ratio.append(len(current_trips))

bins = np.arange(0, 10, 1) # fixed bin size
plt.xlim([0, 10])
plt.hist(ratio, bins=bins, density=True)
plt.title('Histogram of cars being relocated at random times')
plt.xlabel('Cars being relocated')
plt.ylabel('Count')

plt.show()

In [ ]:
((rental[rental['Servicedrive_YN'] == 1]['End_Datetime_Local'] - rental[rental['Servicedrive_YN'] == 1]['Start_Datetime_Local']).dt.total_seconds()/60).hist(bins=1000)
plt.xlim([0, 120])
plt.show()

In [ ]:
ratio = []
for _ in tqdm(range(100)):
    timepoint = time_start + timedelta(seconds=total_time.total_seconds()*random.uniform(0,1))
    #filtered_rentals = rental[rental['End_Datetime_Local'] <= timepoint].drop('Revenue_Net', axis=1)
    #filtered_rentals = filtered_rentals.sort_values(by='End_Datetime_Local').drop_duplicates(subset='Vehicle_Number_Plate', keep='last') # Keep the last location
    current_trips = rental[(rental['Start_Datetime_Local'] <= timepoint) & (rental['End_Datetime_Local'] > timepoint) & (rental['Servicedrive_YN']==0)] # Cars in use
    ratio.append(len(current_trips))

bins = np.arange(0, 200, 1) # fixed bin size
plt.xlim([0, 200])
plt.hist(ratio, bins=bins, density=True)
plt.title('Histogram of cars in random times')
plt.xlabel('Cars in use')
plt.ylabel('Count')

plt.show()

In [ ]:
rental['After_reloc'] = rental.groupby('Vehicle_Number_Plate')['Servicedrive_YN'].shift()

In [ ]:
plt.hist(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0)]['Revenue_Net'], bins=range(0,200,10), density=True, alpha=0.5, label='All revenues')
plt.hist(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0) & (rental['After_reloc']==1)]['Revenue_Net'], bins=range(0,200,10), density=True, alpha=0.5, label='Revenues after relocation')
plt.hist(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0) & (rental['After_reloc']==0)]['Revenue_Net'], bins=range(0,200,10), density=True, alpha=0.5, label='Revenues without relocation')
plt.legend()
plt.show()
print('Mean total:', '{:2.2f}'.format(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0)]['Revenue_Net'].mean()))
print('Mean after relocation:', '{:2.2f}'.format(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0) & (rental['After_reloc']==1)]['Revenue_Net'].mean()))
print('Mean without relocation:', '{:2.2f}'.format(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0) & (rental['After_reloc']==0)]['Revenue_Net'].mean()))
print('Increase:', '{:2.2%}'.format(rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0) & (rental['After_reloc']==1)]['Revenue_Net'].mean()/rental[(rental['Servicedrive_YN']==0) & (rental['Revenue_Net']!=0)]['Revenue_Net'].mean()-1))

In [ ]:
print('Percentage of rentals per zone')
rental.groupby('Start_Zone_Name')['Vehicle_Number_Plate'].count().sort_values(ascending=False)/len(rental)

In [ ]:
print('Distribution of rentals unassigned to areas')
start_map_heat = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
folium.plugins.HeatMap(rental[rental['Start_Zone_Name']=='-'].loc[:,['Start_GPS_Latitude', 'Start_GPS_Longitude']], name=None, min_opacity=0.5, max_zoom=18, radius=10,
blur=8, gradient=None, overlay=True, control=True, show=True).add_to(start_map_heat)
display(start_map_heat)

In [ ]:
print('Total rentals per zone')
rental.groupby('Virtual_Start_Zone_Name')['Vehicle_Number_Plate'].count().sort_values(ascending=False)

In [ ]:
print('Percentage of rentals per zone')
rental.groupby('Virtual_Start_Zone_Name')['Vehicle_Number_Plate'].count().sort_values(ascending=False)/len(rental)

In [ ]:
plt.bar(x=range(len(virtual_area_centers)), height=rental.groupby('Virtual_Start_Zone_Name')['Vehicle_Number_Plate'].count().sort_values(ascending=False).values/len(rental), alpha=0.7, label='Virtual', width=1)
plt.bar(x=range(len(area_centers)), height=rental.groupby('Start_Zone_Name')['Vehicle_Number_Plate'].count().sort_values(ascending=False).values/len(rental), alpha=0.7, label='Original', width=1)
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
silhouette = pd.read_csv(Path.cwd().parent / 'reports' / 'virtual_area_opt.csv', index_col='0')
silhouette.columns = ['Silhouette']
silhouette.index.names = ['n_clusters']
silhouette.plot()
print(silhouette.iloc[np.argmax(silhouette)].name)

In [ ]:
print('Areas vs. virtual areas')
start_map = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
for name, row in area_centers.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['GPS_Latitude'], row['GPS_Longitude']],
        color="crimson",
        tooltip=name,
        fill=False,
    ).add_to(start_map)
for name, row in virtual_area_centers.iterrows():
    folium.CircleMarker(
        radius=5,
        location=[row['GPS_Latitude'], row['GPS_Longitude']],
        color="blue",
        tooltip=name, 
        fill=True
    ).add_to(start_map)
display(start_map)

In [ ]:
hour_start = 5
print('Distribution of origin service drives at {:2n}'.format(hour_start))
start_map_heat = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
folium.plugins.HeatMap(rental[(rental['Servicedrive_YN']==1) & (rental['Start_Datetime_Local'].dt.hour==hour_start)].loc[:,['Start_GPS_Latitude', 'Start_GPS_Longitude']], name=None, min_opacity=0.5, max_zoom=18, radius=10,
blur=8, gradient=None, overlay=True, control=True, show=True).add_to(start_map_heat)
display(start_map_heat)

In [ ]:
print('Distribution of destination service drives at {:2n}'.format(hour_start))
start_map_heat = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
folium.plugins.HeatMap(rental[(rental['Servicedrive_YN']==1) & (rental['Start_Datetime_Local'].dt.hour==hour_start)].loc[:,['End_GPS_Latitude', 'End_GPS_Longitude']], name=None, min_opacity=0.5, max_zoom=18, radius=10,
blur=8, gradient=None, overlay=True, control=True, show=True).add_to(start_map_heat)
display(start_map_heat)

In [ ]:
def plot_service_drives(i, fmap, quatity:int=100, colour='crimson'):
    origin_area = virtual_area_centers.index[i]
    rent_locs = rental[rental['Virtual_Start_Zone_Name']==origin_area].iloc[:quatity]
    folium.CircleMarker(location=[virtual_area_centers.loc[origin_area,'GPS_Latitude'], virtual_area_centers.loc[origin_area, 'GPS_Longitude']], color="blue", tooltip=i, fill=True).add_to(start_map)
    for _, row in rent_locs.iterrows():
        folium.Circle(
            radius=10,
            location=[row['End_GPS_Latitude'], row['End_GPS_Longitude']],
            color=colour,
            fill=False,
        ).add_to(start_map)
    display(start_map)

In [ ]:
start_map = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')
plot_service_drives(20, start_map)

In [ ]:
def plot_rent_locs(i, fmap, quatity:int=500, colour='crimson'):
    area = virtual_area_centers.index[i]
    rent_locs = rental[rental['Virtual_Start_Zone_Name']==area].loc[:,['Start_GPS_Latitude', 'Start_GPS_Longitude']].iloc[:quatity]
    for _, row in rent_locs.iterrows():
        folium.Circle(
            radius=10,
            location=[row['Start_GPS_Latitude'], row['Start_GPS_Longitude']],
            color=colour,
            fill=False,
        ).add_to(start_map)

In [ ]:
start_map = folium.Map([55.6785706133019, 12.594427257404426], zoom_start=12, tiles='Stamen Toner')

plot_rent_locs(34, start_map, colour='red')
plot_rent_locs(0, start_map, colour='green')
plot_rent_locs(22, start_map, colour='yellow')
plot_rent_locs(48, start_map, colour='purple')

for idx, row in virtual_area_centers.iterrows():
    folium.CircleMarker(
        location=[row['GPS_Latitude'], row['GPS_Longitude']],
        color="blue",
        tooltip=idx, 
        fill=True
    ).add_to(start_map)

display(start_map)